In [ ]:
# імпорт бібліотек
import os
from tkinter import filedialog
import cv2
import matplotlib.pyplot as plt
import numpy as np

## Теоретична довідка

> В даний час зріс інтерес до різних методів ідентифікації осіб. Длявирішення
цього завдання необхідно виконати два етапи: виявити особа на фотографії,
виділити і розпізнати його.
При всьому різноманітті різних алгоритмів і методів розпізнавання
зображень, типовий метод розпізнавання складається з трьох компонентів:

- перетворення вихідного зображення в початкове уявлення (може
включати в себе як попередню обробку, так і математичні перетворення,
наприклад обчислення головних компонент);
- виділення ключових характеристик (наприклад береться першіn головних
компонент або коефіцієнтів дискретного косинусного перетворення);
- механізм класифікації (моделювання): кластерна модель, метрика,
нейронна мережа тощо.

> Крім цього, побудова методу розпізнавання спирається на апріорну
інформацію про предметну область (в даному випадку - характеристики особи
людини), і коригується експериментальної інформацією, що з'являється по ходу
розробки методу.

## Допоміжні функції для роботи з картинками

In [ ]:
def show_image(img, title=None):
    """
    Відображує зображення у matplotlib у сірому відтінку.

    Parameters:
    - `img (numpy.ndarray)`: Зображення у форматі NumPy ndarray.
    - `title (str, optional)`: Заголовок для відображення. За замовчуванням - None.

    Returns:
    - `None`
    """
    plt.imshow(img,cmap='gray')
    plt.title(title)
    plt.show()
    
def load_image():
    """
    Функція зчитування зображення.

    Returns:
    - `numpy.ndarray or None`: Зображення у форматі NumPy ndarray або None, якщо завантаження не вдалося.
    """
    file_path = filedialog.askopenfilename()

    if not file_path:
        return None
    if not os.path.isfile(file_path):
        return None

    # Завантажуємо кольорове зображення
    return cv2.imread(file_path)

## `Завдання 1`. <span style="color:lightblue">Розпізнавання обличчя на картинці.</span>

>Завантаження класифікатора каскаду для виявлення облич

In [ ]:

face_cascade_db = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

>зчитування картинки

In [ ]:
img = load_image()
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
show_image(img_gray, "Завантажене фото обличчя")
cv2.waitKey(0)
cv2.destroyAllWindows()

> фото з відміченням знайденого обличчя
> тут в залежності від параметрів **`scaleFactor = 1.1, minNeighbors = 19`** буде змінюватися якість пошуку обличчя

In [ ]:
faces = face_cascade_db.detectMultiScale(img_gray, 1.1, 19)

# Намальовування прямокутників навколо виявлених облич на кольоровому зображенні
for x, y, w, h in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Відображення кольорового зображення з прямокутниками навколо облич
cv2.imshow("Detected Faces", img)

# Очікування натискання клавіші
cv2.waitKey(0)

# Закриття вікна
cv2.destroyAllWindows()

## `Завдання 2`. <span style="color:lightblue">Розпізнавання обличчя на відео.</span>


> вмикаємо зчитування з камери

In [ ]:
cap = cv2.VideoCapture(0)

> нескінчений цикл для зчитування кадрів з відеокамери та у ньому працює цикл котрий буде прямокутники які показиватимуть знайдені обличчя

In [ ]:
while True:
    # Зчитування кадру з відеокамери
    success, video_img = cap.read()

    # Перетворення кадру в відтінки сірого
    img_video_gray = cv2.cvtColor(video_img, cv2.COLOR_BGR2GRAY)

    # Виявлення обличчів на кадрі
    faces = face_cascade_db.detectMultiScale(img_video_gray, scaleFactor=1.1, minNeighbors=19)

    # Намалювання прямокутників навколо виявлених облич
    for x, y, w, h in faces:
        cv2.rectangle(video_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Відображення зображення з прямокутниками навколо облич у вікні
    cv2.imshow("Detected Faces", video_img)

    # Умова виходу з циклу при натисканні клавіші 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        # Закриття вікон OpenCV та виход з циклу
        cv2.destroyAllWindows()
        break

> вивмикаємо камеру після завершення роботи

In [ ]:
cap.release()